In [1]:
solver_grammar = r"""
    ?start: _NL? problem
    problem: ([solveoperation | operation | decl | expr] _NL)+
    decl: symbol ":=" expr
    expr: method
        | symbol
        | literal
    method: symbol "(" parameter ")"
    parameter: [expr | operation] ("," [expr | operation])*
    operation: expr (expr)* OPERATOR expr (expr)*
    solveoperation: (operation | expr) "=" "?"
    symbol: SYMBOL
    literal: LITERAL
    
    SYMBOL: /[a-zA-Z][a-zA-Z0-9]*/
    LITERAL: /-?\d*\.?\d+/
    OPERATOR: /([\/*\-+]|==|\\E|\\X)/
    COMMENT: "(*" /(.|\n|\r)+/ "*)"     
           |  "{" /(.|\n|\r)+/ "}"      
           |  "#" /(.)+/
    
    %import common.INT -> NUMBER
    %import common.NEWLINE -> _NL
    %import common.WS_INLINE
    
    %ignore WS_INLINE
    %ignore COMMENT
"""

In [6]:
from lark import Lark
parser = Lark(solver_grammar)
sample = """
t := triangle(a, b, c)
d := triangle(z, x, y)
perimeter(d, 3)
perimeter(t, 2 * perimeter(d))
dist(a, b) == 2
eq(dist(a, c), 2)
solve(dist(b, c))
"""
sample1 = """
t := triangle(a, b, c)
perimeter(t, 2)
solve(2 * perimeter(t))
"""
sample2 = """
t := triangle(a, b, c)
perimeter(t, 2 * 2)
eq(dist(a, b), 1)
eq(dist(a, c), 2)
solve(dist(b, c))
"""
sample3 = """
t := triangle(a,b,c)
perimeter(t, 5)
dist(a,c) == dist(b,c)
eq(dist(a,b),1)
solve(dist(a,c))
"""
sample4 = """
c := circle(a, r)
area(c, 314)
solve(r)
"""

sample5 = """
    t := triangle(a,b,c)
    c1 := mid(a,b)
    a1 := mid(b,c)
    b1 := mid(c,a)
    t1 := triangle(a1,b1,c1)
    perimeter(t1,13)
    eq(dist(a,b) / dist(b,c),3/4)
    eq(dist(b,c) / dist(c,a),4/6)
    solve(dist(a,b))
"""

sample6 = """
    t := triangle(a,b,c)
    a1 := mid(b,c)
    b1 := mid(c,a)
    m := intersect(a,a1,b,b1)
    solve(dist(a,m)/dist(m,a1))
"""

sample7 = """
    t := triangle(a, b, c) (*
        sdsa*)
    angle(a, c, b, 90) {dsadsad
    asdsadasd}
    dist(a, c) == 3 #dwdwqdwdwq
    dist(c, b) == 4
    h == project(c, line(a, b))
    solve(dist(c, b) * dist(c, h))
"""
sample8 = r"""
    x \E a b
    belongs(p, c, d)
    x1 y1 \X x2 y2
    intersect(x3, y3, x4, y4, f)
"""
sample9 = """
    t := triangle(A, B, C)
    eq(dist(A, B), 3)
    eq(dist(B, C), 4)
    eq(dist(A, C), 5)
    belongs(A, C, D) # Let D belongs to AC
    angularbisector(A, B, C, D) # Angular bisector on <ABC and use D as the second point of the bisector 
    solve(angle(C, B, D))
"""

print(parser.parse(sample9).pretty())

ModuleNotFoundError: No module named 'lark'

In [8]:
from problem import *
from lark import Tree, Token

p = Problem() 

methods = {
    'line': p.line,
    'triangle': p.triangle,
    'circle': p.circle,
    'perimeter': p.perimeter,
    'area': p.area,
    'angle': p.angle,
    'eq': p.eq,
    'dist': p.dist,
    'solve': p.solve,
    'mid': p.midpoint,
    'intersect': p.intersect,
    'altitude': p.altitude,
    'project': p.project,
    'angularbisector': p.angular_bisector,
    'belongs': p.belongs,
    
    '+': p.add,
    '-': p.sub,
    '*': p.mul,
    '/': p.div,
    '==': p.eq,
    '\\X': p.intersect,
    '\\E': p.belongs,
}

def process_expression(root, problem):
    if root.data == 'literal':
        return(float(root.children[0]))
    elif root.data == 'symbol':
        return(str(root.children[0]))
    elif root.data == 'method':
        method = visit_branch(root, problem)
        return(methods.get(method[0], lambda: 'Invalid')(*method[1]))

    #return visit_branch(root)

def visit_branch(root, problem):
    if isinstance(root, Tree):
        params = [visit_branch(child, problem) for child in root.children]
       # print(params)
        if root.data == 'expr' and len(params) == 1:
            return process_expression(root.children[0], problem)
        elif root.data == 'operation':
            res = [visit_branch(child, problem) for child in root.children]
            idx = 0
            for child in root.children:
                if isinstance(child, Token) and child.type == 'OPERATOR':
                    op = res.pop(idx)
                idx += 1
            return (methods.get(op, lambda: 'Invalid')(*res))
        elif root.data == 'solveoperation':
            return (methods.get('solve')(visit_branch(root.children[0], problem)))
        elif root.data == 'decl':
            problem.bind(*params)
        elif root.data == 'literal' or root.data == 'symbol':
            return params[0]
        return params
    if isinstance(root, Token):
        return str(root)
               
visit_branch(parser.parse(sample4), p)[-1]

9.997464891746821